## Purpose

- Async pull latest kitties that are for sale with given price boundaries 
- Should I pull in lots of 2000? Then I can make predictions on! I think so 
- Should also combine the latest pull of id_s with pulling of traits
- create df that is model ready (has latest trends and ETH price)
- how to deal with get_dummies?
- Need latest ETH TO USD
- 80000000000000000 = 0.08 ETH (divide by 1000000000000000000)



In [12]:
import requests
import json
import pandas as pd
import codecs
pd.set_option("max_columns", 30)
import os

In [13]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [14]:
ETH = pd.read_csv("../Data/ETH_price.csv")
ETH.head()

,date,open,high,low,close,volume,marketcap
0,2019-07-06,287.89,295.99,286.67,287.55,7674615025,30707785756
1,2019-07-05,284.38,293.92,282.69,288.00,8723493015,30751916427
2,2019-07-04,303.03,303.96,282.77,284.52,8403513106,30377082394
3,2019-07-03,291.76,303.34,291.40,303.10,9926711979,32356299765
4,2019-07-02,293.54,295.66,272.60,291.60,10618413952,31124332021


In [15]:
trends = pd.read_csv("../Data/trends.csv")
trends.head()

,date,cryptokitties
0,2019-07-04,0.544530
1,2019-07-03,0.877299
2,2019-07-02,0.514278
3,2019-07-01,0.544530
4,2019-06-30,0.423523


In [5]:
url = 'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit=20&offset=53652&orderDirection=asc'
payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False)

parsed = json.loads(response.text)
print(json.dumps(parsed, indent=4, sort_keys=True))

{
    "kitties": [
        {
            "auction": {
                "current_price": "1000000000000000000",
                "duration": "345600000",
                "end_price": "1000000000000000000",
                "end_time": "2018-10-27 16:24:14+00",
                "id": 1032695,
                "seller": {
                    "address": "0xcc34a5df6536a06d195cf22303dd1798c3c70af9",
                    "hasDapper": false
                },
                "start_price": "500000000000000000",
                "start_time": "2018-10-23 16:24:14+00",
                "status": "open",
                "type": "sale"
            },
            "bio": "What's up! I'm #{name}. My friends describe me as stinky and seductive. When my owner isn't watching, I steal their brooches and use them for litter paper. I'm not sorry. I think you'll love me beclaws I have cattitude.",
            "color": "babypuke",
            "created_at": "2018-01-13T11:41:46Z",
            "enhanced_cattributes":

In [6]:
all_urls = []
for i in range(int(53662/100)+ 2):
    all_urls.append(f'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit={100}&offset={i*100}&orderDirection=asc')

In [14]:
round(int(len(all_urls))/10 - 1)

53

## Async getting latest kitties for sale!


In [32]:
import aiofiles
import os
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp


payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

async def download_html(session, url):
    async with session.request('GET', url, headers = headers, data = payload, allow_redirects=False) as res:
        filename = f'html/output_{len(os.listdir("html/"))}.html'
        print(len(os.listdir("./html")), end = "\r")
        async with aiofiles.open(filename, 'wb') as f:
            while True:
                chunk = await res.content.read(1024)
                if not chunk:
                    break
                await f.write(chunk)

        return await res.release()
    
async def main(url):
    connector = aiohttp.TCPConnector(limit=60)
    async with aiohttp.ClientSession(connector=connector) as session: #aiohttp.ClientSession()
        await download_html(session, url)

# urls = ['https://public.api.cryptokitties.co/v1/kitties/' + str(i) for i in range(100)] # a big list of websites to scrape

for i in range(int(len(all_urls)/10)+ 1):
    urls = all_urls[i*10: i*10 + 10]
    print('step: ' + str(i) + '   Number of files: ' + str(len(os.listdir("html/"))))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        asyncio.gather(*(main(url) for url in urls))
    )

step: 0   Number of files: 1
step: 1   Number of files: 11
step: 2   Number of files: 21
step: 3   Number of files: 31
step: 4   Number of files: 41
step: 5   Number of files: 51
step: 6   Number of files: 61
step: 7   Number of files: 71
step: 8   Number of files: 81
step: 9   Number of files: 91
step: 10   Number of files: 101
step: 11   Number of files: 111
step: 12   Number of files: 121
step: 13   Number of files: 130
step: 14   Number of files: 140
step: 15   Number of files: 150
step: 16   Number of files: 160
step: 17   Number of files: 170
step: 18   Number of files: 180
step: 19   Number of files: 190
step: 20   Number of files: 200
step: 21   Number of files: 210
step: 22   Number of files: 220
step: 23   Number of files: 230
step: 24   Number of files: 240
step: 25   Number of files: 250
step: 26   Number of files: 260
step: 27   Number of files: 270
step: 28   Number of files: 280
step: 29   Number of files: 290
step: 30   Number of files: 300
step: 31   Number of files: 3

### ERROR HANDLING!

In [24]:
errors = []
for i in os.listdir("html/"):
    file = codecs.open("html/" + str(i), 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
    except:
        pass
    
    try:
        parsed['code']
        print(i)
        errors.append(i)
    except:
        pass
        

In [23]:
payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

for e in errors:
    limit = 100
    offset = (int(e[7:].split(".")[0])-1)*100

    url = f'https://public.api.cryptokitties.co/v1/kitties?include=sale&price=80000000000000000-1000000000000000000&orderBy=current_price&limit={100}&offset={offset}&orderDirection=asc'
    response = requests.request('GET', url, headers = headers, data = payload, allow_redirects=False).text

    file = open("html/" + e, 'w', encoding="utf-8")
    file.write(response)
    file.close()

    print(e)

output_389.html


### PUTTING INTO DF

In [16]:
def sorting(val): 
    return (int(val[7:].split(".")[0])-1)  

dirs = sorted(os.listdir("html/"), key = sorting)
dirs

['output_0.html',
 'output_1.html',
 'output_2.html',
 'output_3.html',
 'output_4.html',
 'output_5.html',
 'output_6.html',
 'output_7.html',
 'output_8.html',
 'output_9.html',
 'output_10.html',
 'output_11.html',
 'output_12.html',
 'output_13.html',
 'output_14.html',
 'output_15.html',
 'output_16.html',
 'output_17.html',
 'output_18.html',
 'output_19.html',
 'output_20.html',
 'output_21.html',
 'output_22.html',
 'output_23.html',
 'output_24.html',
 'output_25.html',
 'output_26.html',
 'output_27.html',
 'output_28.html',
 'output_29.html',
 'output_30.html',
 'output_31.html',
 'output_32.html',
 'output_33.html',
 'output_34.html',
 'output_35.html',
 'output_36.html',
 'output_37.html',
 'output_38.html',
 'output_39.html',
 'output_40.html',
 'output_41.html',
 'output_42.html',
 'output_43.html',
 'output_44.html',
 'output_45.html',
 'output_46.html',
 'output_47.html',
 'output_48.html',
 'output_49.html',
 'output_50.html',
 'output_51.html',
 'output_52.html',
 'o

In [17]:
id_s = []
for d in dirs:
    file= codecs.open("html/"+ d, 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
    except:
        pass
    print(d + " DONE!", end = "\r")
    
    for i in range(100):
        try: 
            id_s.append(parsed['kitties'][i]['id'])
        except:
            print(str(d), end = "\r")

## NEED TO REMEMBER HOW I GOT INTO DF

In [27]:
data = pd.DataFrame(id_s, columns = ['ID'])

In [28]:
data.head()

,ID
0,1532879
1,1460942
2,1549546
3,1513584
4,1132878


In [25]:
# data = pd.read_csv("Latest.csv")
# data.head()

In [29]:

data['USD'] = ''
data['time'] = ''
# Random
data['generation'] = ''
data['color'] = ''
data['is_fancy'] = ''
data['is_exclusive'] = ''
data['fancy_type'] = ''
data['is_prestige'] = ''
data['prestige_type'] = ''
data['prestige_ranking'] = ''
data['fancy_ranking'] = ''
data['prestige_time_limit'] = ''

# Enhanced cattributes
data['enhanced_cattributes_eyes'] = '' 
data['enhanced_cattributes_pattern'] = '' 
data['enhanced_cattributes_body'] = '' 
data['enhanced_cattributes_mouth'] = '' 
data['enhanced_cattributes_colorprimary'] = '' 
data['enhanced_cattributes_colorsecondary'] = '' 
data['enhanced_cattributes_colortertiary'] = '' 
data['enhanced_cattributes_coloreyes'] = '' 

# Status
data['status_cooldown_index'] = ''
data['status_is_ready'] = ''
data['status_is_gestating'] = ''

# Purrs
data['purrs_count'] = ''
data['purrs_is_purred'] = ''

# watchlist
data['watchlist_count'] = ''
data['watchlist_is_watchlisted'] = ''


# resorting columns!
# data = data[['time', 'ID', "USD"] + list(data.columns)[:-2][1:]]

In [40]:
data[['time', 'ID', "USD"] + list(data.columns)[3:]]

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,,1532879,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,1460942,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,1549546,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,1513584,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,1132878,,,,,,,,,,,,,,,,,,,,,,,,,,
5,,1502777,,,,,,,,,,,,,,,,,,,,,,,,,,
6,,1355806,,,,,,,,,,,,,,,,,,,,,,,,,,
7,,1540985,,,,,,,,,,,,,,,,,,,,,,,,,,
8,,1116582,,,,,,,,,,,,,,,,,,,,,,,,,,
9,,1313291,,,,,,,,,,,,,,,,,,,,,,,,,,


In [30]:
data.head()

,ID,USD,time,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,1532879,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1460942,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1549546,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1513584,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,1132878,,,,,,,,,,,,,,,,,,,,,,,,,,,


## NEED TO FIGURE THIS OUT!

In [ ]:
k = 0
for d in dirs:
    file= codecs.open("html/"+ d, 'r',  encoding="utf8")
    try:
        parsed = json.loads(file.read())
    except:
        pass
    
    parsed = parsed['kitties']
    
    for i in range(100):
        try: 
            # Price
            data.iloc[k, 2] =  int(parsed[i]['auction']['current_price'])/1000000000000000000
            
            # Random
            data.iloc[k, 3] = parsed[i]['generation']        
            data.iloc[k, 4] = parsed[i]['color']
            data.iloc[k, 5] = parsed[i]['is_fancy']
            data.iloc[k, 6] = parsed[i]['is_exclusive']
            data.iloc[k, 7] = parsed[i]['fancy_type']
            data.iloc[k, 8] = parsed[i]['is_prestige']
            data.iloc[k, 9] = parsed[i]['prestige_type']
            data.iloc[k, 10] = parsed[i]['prestige_ranking']
            data.iloc[k, 11] = parsed[i]['fancy_ranking']
            data.iloc[k, 12] = parsed[i]['prestige_time_limit']

            # Enhanced cattributes
            if parsed[i]['enhanced_cattributes'] != []:
                data.iloc[k, 13] = parsed[i]['enhanced_cattributes'][0]['description']
                data.iloc[k, 14] = parsed[i]['enhanced_cattributes'][1]['description']
                data.iloc[k, 15] = parsed[i]['enhanced_cattributes'][2]['description']
                data.iloc[k, 16] = parsed[i]['enhanced_cattributes'][3]['description']
                data.iloc[k, 17] = parsed[i]['enhanced_cattributes'][4]['description']
                data.iloc[k, 18] = parsed[i]['enhanced_cattributes'][5]['description']
                data.iloc[k, 19] = parsed[i]['enhanced_cattributes'][6]['description'] 
                data.iloc[k, 20] = parsed[i]['enhanced_cattributes'][7]['description']

            # Status
            data.iloc[k, 21] = parsed[i]['status']['cooldown_index']
            data.iloc[k, 22] = parsed[i]['status']['is_ready']
            data.iloc[k, 23] = parsed[i]['status']['is_gestating']

            # Purrs
            data.iloc[k, 24] = parsed[i]['purrs']['count']
            data.iloc[k, 25] = parsed[i]['purrs']['is_purred']

            # watchlist
            data.iloc[k, 26] = parsed[i]['watchlist']['count']
            data.iloc[k, 27] = parsed[i]['watchlist']['is_watchlisted']


            print(f"we at: {k} out of 53646. Giving us percent complete of {round(100*k/53646, 5)}%   ", end = "\r")

            if k % 100 == 0:
                data.to_csv("Latest.csv", index = False)
                
            k = k + 1
        except:
            print("\n" + str(d))

we at: 67 out of 53646. Giving us percent complete of 0.12489%   
output_0.html
we at: 106 out of 53646. Giving us percent complete of 0.19759%   
output_1.html


In [10]:
data

NameError: name 'data' is not defined

In [12]:
data.to_csv("Latest.csv", index = False)

In [3]:
import datetime
import pandas as pd

In [4]:
d = pd.read_csv("Latest_kitties.csv")

In [5]:
d.head()

,time,ID,USD,USD.1,time.1,generation,color,is_fancy,is_exclusive,fancy_type,...,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred
0,forgetmenot,1532879,9,9,forgetmenot,False,False,NaN,True,timbers,...,royalpurple,icy,forgetmenot,4,True,False,2,False,0,False
1,sapphire,1460942,17,17,sapphire,False,False,NaN,False,NaN,...,coffee,sully,sapphire,8,True,False,0,False,0,False
2,mintgreen,1549546,5,5,mintgreen,False,False,NaN,False,NaN,...,orangesoda,wolfgrey,cobalt,4,True,False,0,False,0,False
3,dahlia,1513584,4,4,dahlia,False,False,NaN,False,NaN,...,ducky,cottoncandy,lemonade,2,True,False,1,False,0,False
4,sapphire,1132878,2,2,sapphire,False,False,NaN,False,NaN,...,turtleback,patrickstarfish,sapphire,1,True,False,1,False,0,False


In [9]:
d.to_csv("Latest_kitties_{}.csv".format(datetime.datetime.now().isoformat().replace(':', "-")[:-7]), index = False)

In [42]:
pd.read_csv("Latest_kitties_2019-07-07T20-08-25.csv").head(10)

c:\users\dylan\anaconda3\envs\kitties\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4,5,6,7,8,9,12,13,14,15,16,17,18,19,20,22,23,25,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,NaN,1532879,0.08,9.0,forgetmenot,False,False,NaN,True,timbers,141.0,NaN,2019-05-10T06:59:59Z,beard,amur,mekong,swarley,cinderella,royalpurple,icy,forgetmenot,4.0,True,False,2.0,False,0.0,False
1,NaN,1460942,0.08,17.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,confuzzled,chantilly,wonky,ganado,hintomint,coffee,sully,sapphire,8.0,True,False,0.0,False,0.0,False
2,NaN,1549546,0.08,5.0,mintgreen,False,False,NaN,False,NaN,NaN,NaN,NaN,beard,ragdoll,asif,thunderstruck,salty,orangesoda,wolfgrey,cobalt,4.0,True,False,0.0,False,0.0,False
3,NaN,1513584,0.08,4.0,dahlia,False,False,NaN,False,NaN,NaN,NaN,NaN,beard,wonky,savannah,tiger,salty,ducky,cottoncandy,lemonade,2.0,True,False,1.0,False,0.0,False
4,NaN,1132878,0.08,2.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,sphynx,soserious,slyboots,rascal,bananacream,turtleback,patrickstarfish,sapphire,1.0,True,False,1.0,False,0.0,False
5,NaN,1502777,0.08,2.0,olive,False,False,NaN,False,NaN,NaN,NaN,NaN,olive,confuzzled,ragdoll,chronic,ganado,hanauma,bananacream,padparadscha,1.0,True,False,0.0,False,0.0,False
6,NaN,1355806,0.08,2.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,thicccbrowz,totesbasic,happygokitty,koladiviya,cottoncandy,coffee,granitegrey,sapphire,1.0,True,False,1.0,False,0.0,False
7,NaN,1540985,0.08,4.0,forgetmenot,False,False,NaN,False,NaN,NaN,NaN,NaN,cymric,tiger,wiley,wuvme,onyx,wolfgrey,icy,forgetmenot,2.0,True,False,0.0,False,0.0,False
8,NaN,1116582,0.08,3.0,dahlia,False,False,NaN,False,NaN,NaN,NaN,NaN,confuzzled,thicccbrowz,scorpius,koladiviya,cinderella,safetyvest,morningglory,dahlia,1.0,True,False,1.0,False,0.0,False
9,NaN,1313291,0.08,6.0,thundergrey,False,False,NaN,True,reindeer,663.0,NaN,2019-01-08T08:00:00Z,confuzzled,ragdoll,elk,swarley,rorschach,nachocheez,lilac,kalahari,3.0,True,False,0.0,False,0.0,False
